In [1]:

from transformers import RobertaTokenizerFast

In [2]:
tokenizer = RobertaTokenizerFast.from_pretrained('filiberto')

file filiberto/config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
file filiberto/config.json not found
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [3]:
import torch
def mlm(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15)*(tensor >2 )
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i][selection] = 4
    return tensor


In [4]:
from pathlib import Path
paths = [str(x) for x in Path('./data/text/oscar_it').glob('*.txt')]
paths[:5]

['data/text/oscar_it/text_700.txt',
 'data/text/oscar_it/text_2092.txt',
 'data/text/oscar_it/text_2086.txt',
 'data/text/oscar_it/text_714.txt',
 'data/text/oscar_it/text_728.txt']

In [5]:
from  tqdm.auto import tqdm
input_ids = []
mask = []
labels = []

for path in tqdm(paths[:10]):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    sample = tokenizer(lines, max_length=512, padding = 'max_length', truncation=True, return_tensors='pt')
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone()))



100%|██████████| 10/10 [00:37<00:00,  3.75s/it]


In [6]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [7]:
input_ids[0][:10]

tensor([    0,  4105,  2157,   324,   515,   354,   343, 13662,   306,   495])

In [8]:
labels[0][:10]

tensor([    0,  4105,  2157,   324,   515,   354,   343, 13662,   306,   495])

In [9]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings['input_ids'])
    def __getitem__(self, i):
        return { key : tensor[i] for key, tensor in self.encodings.items()}
        


In [11]:
dataset = Dataset(encodings)

In [12]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [13]:
from transformers import RobertaConfig


In [14]:
config = RobertaConfig(
    vocab_size = tokenizer.vocab_size,
    max_position_embeddings = 514,
    hidden_size = 768,
    num_hidden_layers = 6,
    type_vocab_size = 1,
)

In [15]:
from transformers import RobertaForMaskedLM

In [16]:
model = RobertaForMaskedLM(config)


In [17]:
device = torch.device('cpu')

In [18]:
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [19]:
from transformers import AdamW

In [20]:
optim = AdamW(model.parameters(), lr=1e-4)

In [21]:
from tqdm.auto import tqdm

In [ ]:


epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
